<a href="https://colab.research.google.com/github/slapazromero/EjercicioNumpy/blob/main/Ejercicio_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
from random import randint

In [ ]:
class ImagenNumPy:

  @staticmethod
  def crear_imagen(alto = 100, ancho = 100, color = [0, 0, 0], pintar = True):
    data = np.zeros((alto, ancho, len(color)), dtype = np.uint8)
    data[0:alto, 0:ancho] = color
    if pintar:
      return Image.fromarray(data)
    else:
      return data

  @staticmethod
  def crear_collab(elementos_h = 5, elementos_v = 3, alto = 100, ancho = 100, pintar = True):
    for y in range(0, elementos_v):
      fila = ImagenNumPy.crear_imagen(alto, ancho, (randint(0, 255), randint(0, 255), randint(0, 255)), False)
      ult = False
      for x in range(0, elementos_h - 1):
        fila = np.hstack([fila, ImagenNumPy.crear_imagen(alto, ancho, (randint(0, 255), randint(0, 255), randint(0, 255)), False)])
        if 'data' in locals() :
          data = np.vstack([data, fila])
        else:
          data = np.vstack([fila,])
    if pintar:
      return Image.fromarray(data)
    else:
      return data

  @staticmethod
  def redimensionar_imagen(data, ancho, alto):
    img = Image.fromarray(data)
    img = img.resize(ancho, alto)
    return np.asarray(img)


In [ ]:
ImagenNumPy.crear_imagen(200, 100, [145, 255, 0])

In [ ]:
ImagenNumPy.crear_collab()

ValueError: ignored